# Bayesian Model Comparison: WAIC and LOO for Rheological Models

This notebook demonstrates how to use Bayesian model comparison techniques (WAIC and LOO) to objectively select the best rheological model for your data.

## Learning Objectives

After completing this notebook, you will be able to:
- Understand when model comparison is necessary vs optional
- Compute WAIC (Widely Applicable Information Criterion) for model selection
- Compute LOO (Leave-One-Out cross-validation) via Pareto-smoothed importance sampling
- Interpret ELPD (Expected Log Predictive Density) differences
- Use `az.compare()` for automated model ranking
- Avoid overfitting through penalized likelihood metrics
- Make principled model selection decisions with uncertainty quantification

## Prerequisites

- Understanding of Bayesian inference (`01-bayesian-basics.ipynb`)
- Familiarity with Maxwell and Zener models
- Basic understanding of likelihood and model fit

**Estimated Time:** 35-40 minutes

In [ ]:
# Google Colab Setup - Run this cell first!
# Skip if running locally with rheojax already installed

import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Install rheojax and dependencies
    !pip install -q rheojax
    
    # Colab uses float32 by default - we need float64 for numerical stability
    # This MUST be set before importing JAX
    import os
    os.environ['JAX_ENABLE_X64'] = 'true'
    
    print("✓ RheoJAX installed successfully!")
    print("✓ Float64 precision enabled")

## 1. Introduction: The Model Selection Problem

### Why Model Comparison?

Rheological data can often be fit by multiple models. How do we choose?

**Example Scenario:**
- Stress relaxation data shows exponential decay
- **Maxwell model** (2 parameters: G₀, η) fits with R² = 0.98
- **Zener model** (3 parameters: Ge, Gm, η) fits with R² = 0.99
- **Generalized Maxwell** (5 parameters) fits with R² = 0.995

**Naive approach:** Choose model with highest R²
**Problem:** More complex models *always* fit better (overfitting)

### The Overfitting-Underfitting Trade-off

**Underfitting (too simple):**
- Model misses important features in data
- Poor prediction on new data
- High bias

**Overfitting (too complex):**
- Model fits noise instead of signal
- Poor prediction on new data
- High variance

**Goal:** Find simplest model that captures true data-generating process

### Bayesian Information Criteria

Bayesian model comparison uses **penalized likelihood** metrics:
- Balance goodness-of-fit with model complexity
- Penalize additional parameters
- Estimate out-of-sample prediction accuracy

**Two gold-standard metrics:**
1. **WAIC (Widely Applicable IC):** Asymptotically equivalent to cross-validation
2. **LOO (Leave-One-Out CV):** Direct approximation of cross-validation via importance sampling

**Lower is better:** Model with lowest WAIC/LOO has best expected predictive accuracy

## 2. Setup and Imports

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

import warnings

# ArviZ for model comparison
import arviz as az
import matplotlib
import matplotlib.pyplot as plt

# Standard imports
import numpy as np
import pandas as pd
from IPython.display import display

# Rheo imports
from rheojax.models import Maxwell
from rheojax.models import SpringPot
from rheojax.models import Zener

from rheojax.core.jax_config import safe_import_jax

# Safe JAX import
jax, jnp = safe_import_jax()

# Reproducibility
np.random.seed(42)

# Plotting
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

print("✓ Imports successful")

# Suppress matplotlib backend warning in VS Code
warnings.filterwarnings('ignore', message='.*non-interactive.*')


## 3. Generate Synthetic Data from Zener Model

We generate data from a **Zener model** (3 parameters) and test whether Bayesian model comparison correctly identifies it against simpler (Maxwell) and more complex (SpringPot) alternatives.

In [ ]:
# True Zener parameters
Ge_true = 1e4    # Equilibrium modulus (Pa)
Gm_true = 5e4    # Maxwell arm modulus (Pa)
eta_true = 1e3   # Viscosity (Pa·s)
tau_true = eta_true / Gm_true  # Relaxation time (s)

print("True Zener Parameters:")
print(f"  Ge  = {Ge_true:.2e} Pa (equilibrium modulus)")
print(f"  Gm  = {Gm_true:.2e} Pa (Maxwell arm)")
print(f"  η   = {eta_true:.2e} Pa·s")
print(f"  τ   = {tau_true:.4f} s\n")

# Time array
t = np.logspace(-2, 2, 50)  # 0.01 to 100 s

# True Zener relaxation modulus
# G(t) = Ge + Gm * exp(-t / tau)
G_t_true = Ge_true + Gm_true * np.exp(-t / tau_true)

# Add realistic noise (2%)
noise_level = 0.02
noise = np.random.normal(0, noise_level * G_t_true)
G_t_noisy = G_t_true + noise

print(f"Data: {len(t)} points from {t.min():.2f} to {t.max():.2f} s")
print(f"Noise: {noise_level*100:.0f}% relative")

# Visualize
fig = plt.figure(figsize=(10, 6))
plt.loglog(t, G_t_noisy, 'o', markersize=6, alpha=0.7, label='Synthetic data (Zener + noise)')
plt.loglog(t, G_t_true, '--', linewidth=2, alpha=0.5, label='True Zener response')
plt.axhline(Ge_true, color='red', linestyle=':', linewidth=1.5, alpha=0.5, label=f'Equilibrium modulus Ge = {Ge_true:.0e} Pa')
plt.xlabel('Time (s)')
plt.ylabel('Relaxation Modulus G(t) (Pa)')
plt.title('Stress Relaxation Data (Zener Model)')
plt.legend()
plt.grid(True, alpha=0.3, which='both')
plt.tight_layout()
display(fig)
plt.close(fig)

print("\nKey feature: Finite equilibrium modulus (data plateaus at Ge, does not decay to 0)")
print("This is characteristic of Zener model, NOT Maxwell (which decays to 0)")


## 4. Fit Three Competing Models

We fit three models to the data:
1. **Maxwell** (2 parameters: G₀, η) - simpler than true model
2. **Zener** (3 parameters: Ge, Gm, η) - true model
3. **SpringPot** (2 parameters: G₀, α) - different physics (fractional derivative)

Each model undergoes full Bayesian inference (NLSQ → NUTS) to obtain posterior samples.

In [ ]:
print("="*70)
print("FITTING MODEL 1: MAXWELL")
print("="*70)

# Create and configure Maxwell model
model_maxwell = Maxwell()
model_maxwell.parameters.set_bounds('G0', (1e3, 1e7))
model_maxwell.parameters.set_bounds('eta', (1e1, 1e5))

# NLSQ fit
model_maxwell.fit(t, G_t_noisy)

# Bayesian inference
print("Running Bayesian inference (Maxwell)...")
result_maxwell = model_maxwell.fit_bayesian(
    t, G_t_noisy,
    num_warmup=1000,
    num_samples=2000,
    num_chains=1,
    initial_values={
        'G0': model_maxwell.parameters.get_value('G0'),
        'eta': model_maxwell.parameters.get_value('eta')
    }
)

# Convert to InferenceData
idata_maxwell = result_maxwell.to_inference_data()

print("\n✓ Maxwell model fitted")
print(f"  Converged: R-hat = {max(result_maxwell.diagnostics['r_hat'].values()):.4f}")
print(f"  ESS = {min(result_maxwell.diagnostics['ess'].values()):.0f}")

In [ ]:
print("\n" + "="*70)
print("FITTING MODEL 2: ZENER (TRUE MODEL)")
print("="*70)

# Create and configure Zener model
model_zener = Zener()
model_zener.parameters.set_bounds('Ge', (1e2, 1e6))
model_zener.parameters.set_bounds('Gm', (1e3, 1e7))
model_zener.parameters.set_bounds('eta', (1e1, 1e5))

# NLSQ fit
model_zener.fit(t, G_t_noisy)

# Bayesian inference
print("Running Bayesian inference (Zener)...")
result_zener = model_zener.fit_bayesian(
    t, G_t_noisy,
    num_warmup=1000,
    num_samples=2000,
    num_chains=1,
    initial_values={
        'Ge': model_zener.parameters.get_value('Ge'),
        'Gm': model_zener.parameters.get_value('Gm'),
        'eta': model_zener.parameters.get_value('eta')
    }
)

# Convert to InferenceData
idata_zener = result_zener.to_inference_data()

print("\n✓ Zener model fitted")
print(f"  Converged: R-hat = {max(result_zener.diagnostics['r_hat'].values()):.4f}")
print(f"  ESS = {min(result_zener.diagnostics['ess'].values()):.0f}")

In [ ]:
print("\n" + "="*70)
print("FITTING MODEL 3: SPRINGPOT (FRACTIONAL)")
print("="*70)

# Create and configure SpringPot model
model_springpot = SpringPot()
model_springpot.parameters.set_bounds('c_alpha', (1e-3, 1e9))
model_springpot.parameters.set_bounds('alpha', (0.1, 1.0))

# NLSQ fit
model_springpot.fit(t, G_t_noisy)

# Bayesian inference
print("Running Bayesian inference (SpringPot)...")
result_springpot = model_springpot.fit_bayesian(
    t, G_t_noisy,
    num_warmup=1000,
    num_samples=2000,
    num_chains=1,
    initial_values={
        'c_alpha': model_springpot.parameters.get_value('c_alpha'),
        'alpha': model_springpot.parameters.get_value('alpha')
    }
)

# Convert to InferenceData
idata_springpot = result_springpot.to_inference_data()

print("\n✓ SpringPot model fitted")
print(f"  Converged: R-hat = {max(result_springpot.diagnostics['r_hat'].values()):.4f}")
print(f"  ESS = {min(result_springpot.diagnostics['ess'].values()):.0f}")

## 5. Visual Comparison of Model Fits

Before computing information criteria, let's visually assess how well each model fits the data.

In [ ]:
# Generate predictions from each model
t_plot = np.logspace(-2, 2, 200)
G_maxwell = model_maxwell.predict(t_plot)
G_zener = model_zener.predict(t_plot)
G_springpot = model_springpot.predict(t_plot)

# Plot comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Left: Model fits
ax1.loglog(t, G_t_noisy, 'o', markersize=6, alpha=0.7, label='Data', color='black')
ax1.loglog(t, G_t_true, ':', linewidth=2.5, alpha=0.5, label='True (Zener)', color='gray')
ax1.loglog(t_plot, G_maxwell, '-', linewidth=2, label='Maxwell (2 params)', color='#1f77b4')
ax1.loglog(t_plot, G_zener, '-', linewidth=2, label='Zener (3 params)', color='#ff7f0e')
ax1.loglog(t_plot, G_springpot, '-', linewidth=2, label='SpringPot (2 params)', color='#2ca02c')
ax1.set_xlabel('Time (s)', fontweight='bold')
ax1.set_ylabel('Relaxation Modulus G(t) (Pa)', fontweight='bold')
ax1.set_title('Model Comparison', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3, which='both')

# Right: Residuals
res_maxwell = G_t_noisy - model_maxwell.predict(t)
res_zener = G_t_noisy - model_zener.predict(t)
res_springpot = G_t_noisy - model_springpot.predict(t)

ax2.semilogx(t, res_maxwell / G_t_noisy * 100, 'o-', markersize=5, alpha=0.7, label='Maxwell')
ax2.semilogx(t, res_zener / G_t_noisy * 100, 's-', markersize=5, alpha=0.7, label='Zener')
ax2.semilogx(t, res_springpot / G_t_noisy * 100, '^-', markersize=5, alpha=0.7, label='SpringPot')
ax2.axhline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax2.set_xlabel('Time (s)', fontweight='bold')
ax2.set_ylabel('Relative Residual (%)', fontweight='bold')
ax2.set_title('Residual Analysis', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

# Compute RMSE for each model
rmse_maxwell = np.sqrt(np.mean(res_maxwell**2))
rmse_zener = np.sqrt(np.mean(res_zener**2))
rmse_springpot = np.sqrt(np.mean(res_springpot**2))

print("\nRoot Mean Square Error (RMSE):")
print(f"  Maxwell:   {rmse_maxwell:.2e} Pa")
print(f"  Zener:     {rmse_zener:.2e} Pa  {'✓ Best' if rmse_zener < min(rmse_maxwell, rmse_springpot) else ''}")
print(f"  SpringPot: {rmse_springpot:.2e} Pa")
print("\nNote: Zener has lowest RMSE, but this doesn't account for model complexity!")


## 6. Model Comparison: WAIC and LOO

### Understanding Information Criteria

**WAIC (Widely Applicable Information Criterion):**
- Approximates leave-one-out cross-validation
- Penalizes model complexity via effective number of parameters
- Lower WAIC = better expected out-of-sample prediction
- Formula: WAIC = -2 × (lppd - p_WAIC)
  - lppd: log pointwise predictive density
  - p_WAIC: effective number of parameters

**LOO (Leave-One-Out Cross-Validation):**
- Direct approximation of leave-one-out CV via Pareto-smoothed importance sampling (PSIS)
- More robust than WAIC for influential observations
- Lower LOO = better expected out-of-sample prediction
- Provides Pareto k diagnostic for problematic points

**Interpretation:**
- Δ ELPD < 4: Models indistinguishable
- Δ ELPD 4-10: Moderate evidence for better model
- Δ ELPD > 10: Strong evidence for better model

**Note on ArviZ Output:**
- ArviZ reports ELPD (Expected Log Pointwise Predictive Density)
- With `scale='deviance'`: ELPD values are in deviance scale (-2 × log likelihood)
- Column `elpd_diff` shows the difference from the best model (equivalent to Δ WAIC)
- Traditional WAIC = -2 × ELPD (hence 'deviance' scale)

### ArviZ Automated Comparison

ArviZ provides `az.compare()` for automated model ranking with uncertainty quantification.

In [ ]:
# Create dictionary of models for comparison
models = {
    'Maxwell': idata_maxwell,
    'Zener': idata_zener,
    'SpringPot': idata_springpot
}

# Compute WAIC and LOO for all models
print("="*70)
print("BAYESIAN MODEL COMPARISON")
print("="*70)
print("\nComputing WAIC and LOO for all models...\n")

# Suppress WAIC high variance warnings (expected with limited samples)
# When posterior variance of log predictive densities > 0.4, WAIC may be unreliable
# In such cases, LOO (below) is more robust - we compute both for comparison
warnings.filterwarnings('ignore', message='.*posterior variance of the log predictive densities exceeds.*')

comparison = az.compare(models, ic='waic', method='stacking', scale='deviance')
print(comparison)

print("\n" + "-"*70)
print("INTERPRETATION:")
print("-"*70)
print("Columns:")
print("  rank:       Model rank (0 = best)")
print("  elpd_waic:  ELPD (deviance scale: lower is better)")
print("  p_waic:     Effective number of parameters (complexity penalty)")
print("  elpd_diff:  Difference from best model (Δ WAIC in deviance scale)")
print("  weight:     Stacking weights for model averaging")
print("  se:         Standard error of ELPD")
print("  dse:        Standard error of the difference")
print("  warning:    WAIC reliability warning (True = high posterior variance)")
print("\nRules of thumb:")
print("  elpd_diff < 4:  Models indistinguishable")
print("  elpd_diff 4-10: Moderate evidence for better model")
print("  elpd_diff > 10: Strong evidence for better model")
print("\nNote: scale='deviance' means ELPD values are -2×log(likelihood)")
print("\nWARNING COLUMN INTERPRETATION:")
print("  warning=True: Posterior variance of log predictive densities > 0.4")
print("  This can occur with limited samples (2000 samples, 1 chain)")
print("  When this occurs, LOO is more robust than WAIC")
print("  → See LOO results below for more reliable model comparison")
print("="*70)

In [ ]:
# Compute LOO specifically for detailed diagnostics
print("\n" + "="*70)
print("LOO CROSS-VALIDATION WITH PARETO DIAGNOSTICS")
print("="*70)

# Suppress Pareto k warnings (expected for misspecified models like Maxwell)
# High Pareto k indicates influential observations or model misspecification
# We'll examine the detailed diagnostics below to understand which models are affected
warnings.filterwarnings('ignore', message='.*Estimated shape parameter of Pareto distribution is greater than.*')

# Compute LOO for each model
loo_maxwell = az.loo(idata_maxwell)
loo_zener = az.loo(idata_zener)
loo_springpot = az.loo(idata_springpot)

print("\nMaxwell LOO:")
print(loo_maxwell)

print("\nZener LOO:")
print(loo_zener)

print("\nSpringPot LOO:")
print(loo_springpot)

print("\n" + "-"*70)
print("PARETO K DIAGNOSTIC:")
print("-"*70)
print("Pareto k estimates reliability of LOO approximation per data point:")
print("  k < 0.5:  Good (LOO approximation reliable)")
print("  0.5-0.7:  OK (LOO approximation acceptable)")
print("  0.7-1.0:  Bad (LOO approximation unreliable)")
print("  k > 1.0:  Very bad (LOO approximation fails)")
print("\nIf many points have k > 0.7, consider K-fold CV instead.")
print("\nOBSERVATION:")
print("  Maxwell has 1 'very bad' point (k > 1.0) - this is EXPECTED because")
print("  Maxwell is misspecified for Zener-generated data (wrong model).")
print("  High Pareto k indicates influential observations or poor model fit.")
print("  Zener and SpringPot have all good k values (< 0.7).")
print("="*70)

## 7. Visualize Model Comparison

ArviZ provides publication-quality visualizations of model comparison results.

In [ ]:
# Plot model comparison
az.plot_compare(comparison, figsize=(10, 4))
plt.tight_layout()
fig = plt.gcf()  # Get current figure from ArviZ
display(fig)
plt.close(fig)

print("""
INTERPRETATION - Comparison Plot:
- Models ranked from best (top) to worst (bottom)
- CIRCLE: ELPD (expected log predictive density, higher is better)
- ERROR BARS: Standard error of ELPD
- TRIANGLE: In-sample deviance (not penalized for complexity)

What to look for:
✓ Non-overlapping error bars → clear winner
✗ Overlapping error bars → models statistically equivalent

Expected result:
- Zener should rank #1 (true model)
- Maxwell #2 (underfit, misses equilibrium modulus)
- SpringPot #3 (different physics, wrong functional form)
""")


In [ ]:
# Plot Pareto k diagnostics for best model
best_model_name = comparison.index[0]
best_idata = models[best_model_name]

# Map model name to LOO result (needed for plot_khat)
loo_results = {
    'Maxwell': loo_maxwell,
    'Zener': loo_zener,
    'SpringPot': loo_springpot
}
best_loo = loo_results[best_model_name]

print(f"\nPareto k diagnostic for best model ({best_model_name}):")
az.plot_khat(best_loo, figsize=(10, 4))
plt.tight_layout()
fig = plt.gcf()  # Get current figure from ArviZ
display(fig)
plt.close(fig)

print("""
INTERPRETATION - Pareto k Plot:
- Each point represents one data observation
- Y-axis: Pareto k statistic (reliability of LOO approximation)
- Horizontal lines: Thresholds (0.5 good, 0.7 bad)

What to look for:
✓ All points below 0.5: LOO is reliable
⚠ Few points 0.5-0.7: LOO acceptable but check those points
✗ Many points > 0.7: LOO unreliable, use K-fold CV instead

High Pareto k indicates:
- Influential observations (large impact on posterior)
- Outliers or poorly fit regions
- Potential model misspecification
""")

## 8. Model Weights and Averaging

### Stacking Weights

ArviZ computes **stacking weights** - optimal weights for combining model predictions:
- Sum to 1.0
- Higher weight = model contributes more to ensemble
- If one model has weight ≈ 1.0, it's clearly superior
- If weights distributed, consider model averaging

### When to Use Model Averaging

**Use single best model if:**
- Clear winner (elpd_diff > 10)
- Stacking weight > 0.9
- Physical interpretation matters (choose simplest adequate model)

**Use model averaging if:**
- Models statistically indistinguishable (elpd_diff < 4)
- Stacking weights distributed (no model > 0.7)
- Prediction accuracy paramount (not interpretation)

In [ ]:
# Extract stacking weights
weights = comparison['weight'].values
model_names = comparison.index.tolist()

# Visualize weights
fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(model_names, weights, color=['#1f77b4', '#ff7f0e', '#2ca02c'], alpha=0.7, edgecolor='black')
ax.set_ylabel('Stacking Weight', fontweight='bold')
ax.set_title('Model Weights for Ensemble Prediction', fontweight='bold')
ax.set_ylim([0, 1])
ax.grid(True, alpha=0.3, axis='y')

# Annotate weights
for i, (name, w) in enumerate(zip(model_names, weights)):
    ax.text(i, w + 0.03, f'{w:.3f}', ha='center', fontweight='bold')

plt.tight_layout()
display(fig)
plt.close(fig)

print("\nStacking Weights Interpretation:")
print("-" * 50)
for name, w in zip(model_names, weights):
    print(f"  {name:<12}: {w:.4f}  ", end="")
    if w > 0.9:
        print("(dominant model - use this one) ✓")
    elif w > 0.5:
        print("(major contributor)")
    elif w > 0.1:
        print("(minor contributor)")
    else:
        print("(negligible)")

print("\nRecommendation:")
if weights[0] > 0.9:
    print(f"✓ Use {model_names[0]} (clear winner with weight {weights[0]:.3f})")
elif weights[0] > 0.7:
    print(f"✓ Prefer {model_names[0]} (dominant with weight {weights[0]:.3f})")
else:
    print(f"⚠ Consider model averaging (weights distributed across {sum(w > 0.1 for w in weights)} models)")


## 9. Final Model Selection Decision

### Decision Framework

Combine **statistical evidence** with **physical reasoning**:

**Statistical Criteria:**
1. ELPD comparison (elpd_diff values)
2. Stacking weights
3. Pareto k diagnostics

**Physical Criteria:**
1. Model assumptions match material physics?
2. Parameters physically interpretable?
3. Simplicity (Occam's Razor)

**Example Decision Tree:**
```
IF elpd_diff > 10 AND weight > 0.9:
    → Use clear winner
ELIF elpd_diff < 4:
    → Models equivalent, choose simpler model (fewer parameters)
ELIF weights distributed:
    → Consider model averaging for predictions
ELSE:
    → Balance statistical evidence with physical interpretation
```

In [ ]:
print("="*70)
print("FINAL MODEL SELECTION DECISION")
print("="*70)

# Extract comparison metrics
best_model = comparison.index[0]
elpd_diff_second = comparison.loc[comparison.index[1], 'elpd_diff']
weight_best = comparison.loc[best_model, 'weight']

print(f"\nBest Model: {best_model}")
print(f"  ELPD (deviance): {comparison.loc[best_model, 'elpd_waic']:.2f}")
print(f"  Δ ELPD from 2nd: {elpd_diff_second:.2f}")
print(f"  Stacking weight: {weight_best:.3f}")
print(f"  Effective parameters: {comparison.loc[best_model, 'p_waic']:.2f}")

print("\nStatistical Evidence:")
if elpd_diff_second > 10:
    print(f"  ✓ STRONG: Δ ELPD = {elpd_diff_second:.1f} > 10 (clear winner)")
elif elpd_diff_second > 4:
    print(f"  ✓ MODERATE: Δ ELPD = {elpd_diff_second:.1f} in [4, 10] (likely better)")
else:
    print(f"  ⚠ WEAK: Δ ELPD = {elpd_diff_second:.1f} < 4 (models indistinguishable)")

if weight_best > 0.9:
    print(f"  ✓ DOMINANT: Stacking weight {weight_best:.3f} > 0.9")
elif weight_best > 0.7:
    print(f"  ✓ MAJOR: Stacking weight {weight_best:.3f} > 0.7")
else:
    print(f"  ⚠ DISTRIBUTED: Stacking weight {weight_best:.3f} < 0.7")

print("\nPhysical Reasoning:")
if best_model == 'Zener':
    print("  ✓ Data exhibits finite equilibrium modulus (plateau at long times)")
    print("  ✓ Zener model captures this physics with spring-dashpot parallel arm")
    print("  ✓ 3 parameters provide adequate flexibility without overfitting")
elif best_model == 'Maxwell':
    print("  ⚠ Maxwell assumes complete stress relaxation (G→0 as t→∞)")
    print("  ⚠ Data shows equilibrium modulus (G→Ge ≠ 0)")
    print("  ⚠ Model misspecification likely")
elif best_model == 'SpringPot':
    print("  ⚠ SpringPot models power-law relaxation (different physics)")
    print("  ⚠ Data is better described by exponential + constant (Zener)")
    print("  ⚠ May fit numerically but wrong physical interpretation")

print("\n" + "="*70)
print("RECOMMENDATION")
print("="*70)

if best_model == 'Zener' and elpd_diff_second > 4 and weight_best > 0.7:
    print(f"✓✓✓ Use {best_model} model")
    print("\nRationale:")
    print("  - Strong statistical evidence (ELPD difference and stacking weights)")
    print("  - Correct physics (finite equilibrium modulus)")
    print("  - Parsimonious (3 parameters vs generalized models)")
    print("  - All parameters physically interpretable")
else:
    print(f"Use {best_model} model with caveats")
    print(f"Consider collecting more data or testing additional models")

print("\nNext Steps:")
print("  1. Validate on independent test data")
print("  2. Check parameter physical reasonableness")
print("  3. Examine residuals for systematic errors")
print("  4. Consider model averaging if weights distributed")
print("="*70)

## 10. Key Takeaways

### Main Concepts

1. **Bayesian Model Comparison:**
   - Use WAIC/LOO to balance fit quality with model complexity
   - Lower WAIC/LOO = better expected out-of-sample prediction
   - Automatically penalizes overfitting via effective parameter count

2. **Interpretation Guidelines:**
   - **Δ ELPD < 4:** Models statistically indistinguishable → choose simpler
   - **Δ ELPD 4-10:** Moderate evidence for better model
   - **Δ ELPD > 10:** Strong evidence for better model
   - **Stacking weight > 0.9:** Clear winner, use single model
   - **Weights distributed:** Consider model averaging

3. **Pareto k Diagnostics:**
   - Assesses reliability of LOO approximation per data point
   - k < 0.5: Good (LOO reliable)
   - k > 0.7: Bad (LOO unreliable, use K-fold CV)
   - High k indicates influential observations or model misspecification

4. **Decision Framework:**
   - Combine statistical evidence with physical reasoning
   - Prefer simplest model that adequately explains data (Occam's Razor)
   - Ensure parameters are physically interpretable
   - Validate on independent data when possible

### When to Use Model Comparison

**Essential for:**
- ✓ Multiple candidate models with different physics
- ✓ Uncertain about appropriate model complexity
- ✓ Avoiding overfitting with limited data
- ✓ Objective model selection for publication

**Optional for:**
- Well-established model for specific material class
- Only one physically plausible model
- Exploratory analysis (NLSQ screening sufficient)

### Common Pitfalls

1. **Ignoring Physical Constraints:**
   - Statistical best model may violate physics
   - Always check parameter physical reasonableness
   - Simplest adequate model often preferable to complex best fit

2. **Overfitting with Small Datasets:**
   - Complex models can overfit with few data points
   - WAIC/LOO automatically penalize this via p_WAIC
   - If weights distributed, likely insufficient data to distinguish

3. **Comparing Apples to Oranges:**
   - Only compare models fit to **same data**
   - Same likelihood formulation (e.g., Gaussian observation model)
   - Same response variable (don't compare G(t) vs G'(ω) models)

4. **Ignoring Pareto k Warnings:**
   - High Pareto k → LOO unreliable
   - Investigate influential points before trusting LOO
   - May indicate outliers or model misspecification

### ArviZ Functions Summary

```python
# Model comparison (with deviance scale for traditional IC values)
comparison = az.compare(models, ic='waic', scale='deviance')  # or ic='loo'

# Individual metrics
waic = az.waic(idata)
loo = az.loo(idata)

# Visualization
az.plot_compare(comparison)
az.plot_khat(loo)  # Pareto k diagnostic (pass LOO result, not InferenceData)
```

**Note on ArviZ column names:**
- Modern ArviZ (>=0.15.0) uses `elpd_waic`/`elpd_loo` and `elpd_diff`
- With `scale='deviance'`: values are in traditional IC scale (-2 × log likelihood)
- `elpd_diff` in deviance scale is equivalent to Δ WAIC or Δ LOO

## Next Steps

### Apply to Your Data
- Test multiple rheological models on your experimental data
- Use WAIC/LOO to objectively select best model
- Validate selection with physical reasoning

### Advanced Topics
- **[05-uncertainty-propagation.ipynb](05-uncertainty-propagation.ipynb)**: Propagate parameter uncertainty to predictions
- **[advanced/01-multi-technique-fitting.ipynb](../advanced/01-multi-technique-fitting.ipynb)**: Compare models across multiple test modes
- **[advanced/04-fractional-models-deep-dive.ipynb](../advanced/04-fractional-models-deep-dive.ipynb)**: Compare 11 fractional models

### Further Reading
- Vehtari et al. (2017): "Practical Bayesian model evaluation using leave-one-out cross-validation and WAIC"
- McElreath (2020): "Statistical Rethinking" (Chapter 7: Model Comparison)
- ArviZ documentation: https://arviz-devs.github.io/arviz/

---

## Session Information

In [ ]:
import sys

import rheojax

print(f"Python: {sys.version}")
print(f"Rheo: {rheojax.__version__}")
print(f"JAX: {jax.__version__}")
print(f"NumPy: {np.__version__}")
print(f"ArviZ: {az.__version__}")
print(f"JAX devices: {jax.devices()}")